In [ ]:
import DataProvider
prov = DataProvider.DataProvider(True)

In [ ]:
import numpy as np
X = list()
y = list()
trainVal = list()

In [ ]:
while prov.hasNext():
    Xt, yt = prov.next()
    for i in range(len(yt)):
        if yt[i] == 1:
            trainVal.append(i+1)
            
    X.append(Xt)
    y.append(yt)

In [ ]:
X = np.nan_to_num(np.array(X))
y = np.matrix(y)

X.shape, y.shape

In [ ]:
X_train = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
X_train.shape

In [ ]:
prov = DataProvider.DataProvider(False, maxLength = X_train.shape[1])

In [ ]:
X2 = list()
y2 = list()
actualVal = list()

In [ ]:
while prov.hasNext():
    X2t, y2t = prov.next()
    y2t = y2t[1]
    actualVal.append(y2t)
    
    k = np.zeros(5)
    k[y2t - 1] = 1
    
    X2.append(X2t)
    y2.append(k)

In [ ]:
X2 = np.nan_to_num(np.array(X2))
y2 = np.matrix(y2)

X2.shape, y2.shape

In [ ]:
len(X2[0])

In [ ]:
X_test = X2.reshape(X2.shape[0], X.shape[1], X.shape[2], 1)
X_test.shape

In [ ]:
(X_train.shape[1], X_train.shape[2],1)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten,InputLayer,AveragePooling2D, BatchNormalization
from keras.layers import Conv1D

from keras import optimizers

In [ ]:
model = Sequential()

In [ ]:
model.add(InputLayer(input_shape = (X.shape[1], X.shape[2],1)))

model.add(Conv2D(32, (6,6), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (4,4), activation="relu"))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))


model.add(Dense(5, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# model.add(Dense(units=256, activation='relu', input_dim = prov.returnSizeOfEverySpectogram()))
# #model.add(Dropout(0.1))
# model.add(Dense(64, activation='relu'))
# #model.add(Dropout(0.2))
# #model.add(Dense(32, activation='relu'))
# #model.add(Dense(256, activation='relu'))
# model.add(Dense(units=5, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y, epochs=15, batch_size=64, verbose=1, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
loss_and_metrics = model.evaluate(X_test, y2, batch_size=128)

In [ ]:
pred = model.predict(X_test, batch_size=128)

In [ ]:
pr = np.argmax(pred, axis=1)+1
pr , np.array(actualVal)

In [ ]:
correct = [1 if a == b else 0 for (a, b) in zip(pr, actualVal)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ("Accuracy: " + str(accuracy * 100) + "%")

In [ ]:
for i in pred:
    count = sum(i)
    for j in range(len(i)):
        i[j] = i[j] / count

In [ ]:
import copy
countGrade = copy.deepcopy(actualVal)


In [ ]:
for i in range(len(countGrade)):
    ans = actualVal[i]
    countGrade[i] = pred[i][ans-1]

In [ ]:
result = sum(countGrade) / len(countGrade)
result

In [ ]:
str(int(result*100))

In [ ]:
save_the_model = False

In [ ]:
if save_the_model:
    # serialize model to JSON
    model_json = model.to_json()
    with open(str(int(result*100))+'.json', "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights('model'+str(int(result*100))+".h5")
    print("Saved model to disk")